# Bus Bunching with Sampled Trajectories

In [1]:
import sys
import math
import pandas as pd
import altair as alt
import networkx as nx
import numpy as np

sys.path.append(r"..")

from mcroute import Network, StateSpace
import mcroute.matrix as matrix
import mcroute.vector as vector

In [2]:
# Set up our state space
# state_space = StateSpace.from_csv('data/bus/states.csv')
state_space = StateSpace.from_range(-30, 70)
# Have an identity matrix ready and made for our nodes
identity = matrix.identity(state_space)

# Create a network for each model
n = Network(state_space)
edges = pd.read_csv(f"data/bus/edges.csv")
# Let's create our nodes first
nodes = set(edges['from'].tolist()).union(set(edges['to'].tolist()))
for node in nodes:
    n.add_node(str(node), identity)
matrices = []
# Now let's edd the edges
for idx, edge in edges.iterrows():
    mx = matrix.from_state_delta_csv(state_space, f"data/bus/{edge['from']}-{edge['to']}_states.csv")
    acs = matrix.absorbing_classes(mx)
    if len(acs) > 10:
        print(f"Using previous matrix for {edge['from']}-{edge['to']}")
        # We need to do some data smoothing. Let's take the matrix from the previous link
        mx = matrices[-1]
    matrices.append(mx)
    n.add_edge(edge['from'].astype(str), edge['to'].astype(str), mx)

Using previous matrix for 8-9
Using previous matrix for 9-10
Using previous matrix for 10-11
Using previous matrix for 54-55
Using previous matrix for 73-74
Using previous matrix for 74-75
Using previous matrix for 83-84
Using previous matrix for 85-86
Using previous matrix for 90-91
Using previous matrix for 92-93


In [4]:
matrix.steady_state(mx)

array([8.39164006e-15, 1.13957633e-14, 1.34704335e-14, 1.47896691e-14,
       1.64398661e-14, 1.79524464e-14, 1.93710581e-14, 2.09280731e-14,
       2.22317520e-14, 2.35039828e-14, 2.42517716e-14, 2.51638979e-14,
       2.68824919e-14, 2.88444809e-14, 3.01499302e-14, 3.10318464e-14,
       3.17461311e-14, 3.35801016e-14, 3.49687244e-14, 3.60667361e-14,
       3.66960664e-14, 3.79119444e-14, 3.88463201e-14, 3.98779748e-14,
       4.06365733e-14, 4.16066147e-14, 4.26555055e-14, 4.34117579e-14,
       4.40236886e-14, 4.54495635e-14, 4.61632090e-14, 4.68331999e-14,
       4.68087211e-14, 4.74288373e-14, 4.78683925e-14, 4.80010975e-14,
       4.82336106e-14, 4.86201518e-14, 4.94186588e-14, 5.00858618e-14,
       5.03102360e-14, 5.08927431e-14, 5.12921132e-14, 5.16200737e-14,
       5.17259144e-14, 5.15007922e-14, 5.10856677e-14, 5.09791440e-14,
       5.10614200e-14, 5.07346099e-14, 5.07481773e-14, 5.05582002e-14,
       4.99889085e-14, 4.95274908e-14, 4.89102674e-14, 4.83982814e-14,
      

In [14]:

# Adjust matrices sequentially
path = nx.shortest_path(n, source="0", target="97")
p0 = vector.unit(state_space, '0')
for i in range(len(path)-1):
    p = path[:i+1]
    vecs = n.traverse(p, p0)
    average = np.average(state_space.values, weights=vecs[-1])
    if average > 1.5:
        n.edges[str(i), str(i+1)]['matrix'] = matrix.shift(n.edges[str(i), str(i+1)]['matrix'], idx=-1)
    if average < -1.5:
        n.edges[str(i), str(i+1)]['matrix'] = matrix.shift(n.edges[str(i), str(i+1)]['matrix'], idx=1)

In [15]:
v = vecs[-1]
df = pd.DataFrame(v, columns=['prob']).reset_index()
df['index'] = df['index'] + state_space.values[0]
alt.Chart(df).mark_line().encode(
    alt.X('index:O'),
    alt.Y('prob:Q')
).properties(
    width = 1200,
    height = 600
)

alt.Chart(...)

In [16]:
stats = []
means = []
stds = []
for v in vecs:
    average = np.average(state_space.values, weights=v)
    # Fast and numerically precise:
    variance = np.average((state_space.values-average)**2, weights=v)
    std = math.sqrt(variance)
    means.append(average)
    stds.append(std)
allstats = pd.DataFrame(zip(means, stds), columns=['mean', 'std']).reset_index()
allstats.columns = ['element', 'Mean', 'Standard Deviation']
allstats = allstats[allstats.element % 2 == 0]
allstats['element'] = allstats.element/2

In [17]:
alt.Chart(allstats.melt(id_vars=['element'])).mark_line().encode(
    alt.X('element:O', title='Stop'),
    alt.Y('value:Q', title='Schedule Deviation (min)'),
    color=alt.Color('variable:N', title='Measure')
).properties(
    width = 900,
    height = 300,
    title = 'Modelled Mean and Standard Deviation of Schedule Deviation on Route 3 in Calgary, Fall 2015'
).configure(font='Raleway').configure_view(strokeWidth=0).configure_axis(grid=False, domain=False)

alt.Chart(...)

## Generating Trajectories to Analyze Bus Bunching

In [42]:
# We generate a difference trajectory by comparing pairs of trajectories
# Quick reset of matrices
for node, data in n.nodes(data=True):
        data['matrix'] = identity
bunch_threshold = -10
bunch_count = 0
for run in range(1000):
    bunch = False
    t = n.trajectories(path, p0, n=2, smoothing=5)
    for i in range(len(t[0])):
        if t[1][i]-t[0][i] < bunch_threshold:
            bunch = True
    if bunch == True:
        bunch_count += 1
bunch_count

950

In [43]:
t2 = n.trajectories(path, p0, n=2, smoothing=5)
dt = []
for i in range(len(t2[0])):
    dt.append(t2[0][i]-t2[1][i])
#     print(t2[0][i]-t2[1][i])

# for tra in t2:
print('.'.join([str(int(i)) for i in t2[0]]))
print()
print('.'.join([str(int(i)) for i in t2[1]]))
print()
print('.'.join([str(int(i)) for i in dt]))

0.0.0.1.0.0.0.1.0.0.1.2.1.1.0.1.4.2.0.4.2.1.4.4.0.-1.-2.2.2.2.-1.1.1.3.-2.3.4.0.-2.0.3.5.0.3.7.2.2.-2.0.0.-3.0.-5.-3.2.2.1.6.5.0.4.1.0.-2.-4.1.6.2.-2.3.-2.3.2.-2.1.-4.-7.-7.-7.-7.-2.-3.-2.3.0.5.10.5.1.1.-4.-9.-4.-2.3.0.5.1.-1.1.4.-1.4.-1.4.5.0.-3.2.-3.2.3.6.1.-4.-2.-1.-4.1.-1.4.-1.4.5.1.6.1.-4.1.-4.-4.-1.-3.-5.-4.1.6.2.3.-2.-4.1.-4.1.3.-2.3.-2.2.7.2.7.2.2.7.12.11.6.10.5.0.5.3.7.12.7.2.6.1.5.0.-2.-5.-10.-5.0.-5.-3.-7.-2.-5.-5.-4.-9.-4.1.-4.-5.-1.4.1.-2.3.2.-3

0.0.1.2.1.3.1.1.0.0.1.0.3.2.0.2.0.1.0.3.3.-1.0.1.4.4.2.1.1.1.0.3.0.2.0.-1.0.-2.3.1.5.1.-1.-1.-2.2.-1.-1.-3.2.5.0.-2.0.-1.-1.1.-4.1.0.-1.-4.-2.3.-1.1.0.-3.2.2.-1.-2.2.-3.-5.-10.-5.-2.0.-5.0.1.0.-1.1.5.10.5.6.1.2.-3.0.5.3.3.3.-2.3.-2.3.4.9.5.0.0.5.0.-5.-5.-2.-7.-4.-6.-1.-3.1.4.-1.0.2.-3.1.-2.3.8.3.1.3.8.5.5.0.5.9.4.2.6.2.7.2.-3.2.1.-4.-2.-3.2.3.2.5.10.13.8.5.4.1.0.-3.2.-3.-4.-5.0.-5.0.5.2.-3.2.4.3.2.-3.0.-5.-6.-10.-5.0.3.5.0.1.6.5.6.8.3.6.1.6.1.-4.-7

0.0.-1.-1.-1.-3.-1.0.0.0.0.2.-2.-1.0.-1.4.1.0.1.-1.2.4.3.-4.-5.-4.1.1.1.-1.-2.1.1.

In [44]:
# tp = matrix.from_csv(state_space, 'data/bus/tp_matrix.csv')
bbs = []
for thres in range(-10, 10):
    tp = matrix.truncate_below(state_space, identity, f"{thres}")
#     for r in tp:
#         print(''.join([str(int(bool(i))) for i in r]))
    # identity.shape
    for node, data in n.nodes(data=True):
        data['matrix'] = tp
    bunch_count = 0
    for run in range(1000):
        bunch = False
        t = n.trajectories(path, p0, n=2, smoothing=5)
        for i in range(len(t[0])):
            if t[1][i]-t[0][i] < bunch_threshold:
                bunch = True
        if bunch == True:
            bunch_count += 1
    bbs.append([thres, bunch_count])
bdf = pd.DataFrame(bbs, columns=['thres', 'pct'])
bdf

,thres,pct
0,-10,939
1,-9,927
2,-8,905
3,-7,898
4,-6,855
5,-5,804
6,-4,738
7,-3,680
8,-2,580
9,-1,564


In [45]:
bdf = pd.DataFrame(bbs, columns=['thres', 'pct'])
bdf['pct'] = bdf['pct']/10
main = alt.Chart(bdf).mark_line().encode(
    alt.X('thres:Q', title='Slack Time at Time Point (min)'),
    alt.Y('pct:Q', title='Bus Bunching Incidence (%)')
)

rule = alt.Chart(pd.DataFrame({'y': [97]})).mark_rule(color='#F58426').encode(y='y')


(rule + main).properties(
    width=600,
    height=200
).configure(font='Raleway').configure_view(strokeWidth=0).configure_axis(grid=False, domain=False)

alt.LayerChart(...)

,y,x,z
0,0,0,0.000000
1,1,0,0.000000
2,2,0,0.000000
3,3,0,0.000000
4,4,0,0.000000
...,...,...,...
395,15,19,0.001318
396,16,19,0.021429
397,17,19,0.139069
398,18,19,0.405713


alt.Chart(...)